In [56]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [57]:
#original_df = pd.read_csv("GlobalTemperatures_byYear.csv").drop(columns=["Unnamed: 0"])
#original_df.head()

In [58]:
#original_df.info()

In [59]:
#original_df.dropna(inplace = True)

In [60]:
#def get_temperature_diff_between(df_by_year,start_date,end_date):
   # df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
     #                   (df_by_year["year"] <= end_date)].groupby(["Country","continent"]).mean().reset_index()
    
  #  return df

In [61]:
df = pd.read_csv("Correlation.csv")

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7158 entries, 0 to 7157
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     7158 non-null   int64  
 1   year                           7158 non-null   int64  
 2   Country                        7158 non-null   object 
 3   continent                      7158 non-null   object 
 4   AverageTemperature             7158 non-null   float64
 5   AverageTemperatureUncertainty  7158 non-null   float64
 6   Latitude                       7158 non-null   float64
 7   Longitude                      7158 non-null   float64
 8   GHG                            7158 non-null   float64
 9   temp_diff                      7158 non-null   float64
 10  Code                           7158 non-null   object 
 11  gdp                            7158 non-null   float64
 12  gdp_test                       7158 non-null   f

In [63]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country","continent"]).agg({"gdp":np.sum, "AverageTemperature": np.mean, "GHG": np.sum, "temp_diff": np.sum}).reset_index()
        
    return df

In [64]:
df = get_temperature_diff_between(df,1960,2001)
df.head()

,Country,continent,gdp,AverageTemperature,GHG,temp_diff
0,AFGHANISTAN,Asia,4.110511e+10,14.078186,3.311163e+08,0.707500
1,ALBANIA,Europe,4.009485e+10,13.134764,3.119427e+09,1.179500
2,ALGERIA,Africa,1.311925e+12,23.382044,3.237042e+10,1.390750
3,ANDORRA,Europe,2.124760e+10,11.679135,3.347064e+07,1.331250
4,ANGOLA,Africa,1.367624e+11,22.355035,2.340391e+09,0.533583


In [65]:
df["gdp_test"] = df["gdp"]*2

In [67]:
# This plot is for all years

fig = px.scatter(df, x="temp_diff", y="GHG", color="continent", template="simple_white", hover_name="Country",
                 size = "gdp_test",size_max=80,
                labels={
                     "temp_diff": "Increase in average temperature",
                     "GHG": "Greenhouse Gases Emissions"

                 },)
fig.show(renderer = "browser")

In [26]:
#original_df.to_csv("Correlation.csv")

In [55]:
fig_1 = px.scatter(df, x="temp_diff", y="GHG", size="gdp_test", color="continent",
           hover_name="Country", log_x=False, size_max=100,
                  labels={
                     "temp_diff": "Increase in average temperature between 1960 and 2001 years",
                     "GHG": "Greenhouse Gases Emissions"
                      }, custom_data=['Country', 'gdp'])


fig_1.update_traces(
    hovertemplate="<br>".join([
        "Country: %{customdata[0]}",
        "GDP: %{customdata[1]}",
        "Increase in average temperature: %{x}",
        "Greenhouse Gases Emissions: %{y}",
    ])
)


fig_1.show(renderer = "browser")